In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

#import dask.dataframe as dd

from datetime import datetime

In [2]:
# cluster.close()
# client.close()

In [3]:
#from dask.distributed import Client, LocalCluster
#cluster = LocalCluster()
#client = Client(cluster)
# client = Client(n_workers=4, threads_per_worker=4, memory_limit='12GB')
# client

In [4]:
import keras 

from keras.preprocessing import sequence
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, LeakyReLU
from keras.layers import Embedding
from keras.layers import Concatenate, Flatten, Reshape, Lambda
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, LSTM
from keras.utils import plot_model
from keras import backend as K

from keras.datasets import imdb

Using TensorFlow backend.


In [5]:
#K.tensorflow_backend._get_available_gpus()

Helpful links:
1. Inverse of np.log1p: https://stackoverflow.com/questions/50049891/what-is-the-inverse-of-numpys-log1p
2. Building parallel keras model: https://stackoverflow.com/questions/43151775/how-to-have-parallel-convolutional-layers-in-keras
3. How to make parallel keras models: https://datascience.stackexchange.com/questions/39407/how-to-make-two-parallel-convolutional-neural-networks-in-keras
4. 2 parallel keras layers: https://stackoverflow.com/questions/51546075/two-parallel-conv2d-layers-keras
5. Setting up Conv1D and LSTM: https://stackoverflow.com/questions/51344610/how-to-setup-1d-convolution-and-lstm-in-keras
6. Take average of LSTM hidden states using Lambda: https://stackoverflow.com/questions/51479940/average-channels-of-convolutional-layer-keras
7. Preparing categorical variable for neural networks: https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/
8. Tutorial for using keras for time series (no generators used): https://medium.com/@jdwittenauer/deep-learning-with-keras-structured-time-series-37a66c6aeb28


In [6]:
train = pd.read_csv("gs://123test_bucket/train.csv")

In [7]:
reading = train['meter_reading']
train['meter_reading'] = np.log1p(train['meter_reading'])

scaler1 = MinMaxScaler()
train['meter_reading'] = scaler1.fit_transform(X=np.reshape(train['meter_reading'].values, (-1, 1))).reshape(len(train),)

In [8]:
train['meter_reading'].mean()

0.2452794762030242

#### Reading Test Data

In [9]:
test = pd.read_csv("gs://123test_bucket/test.csv")

#### Obtaining Month, Day of the Week, and Hour

In [10]:
train.timestamp = pd.to_datetime(train.timestamp)

test.timestamp = pd.to_datetime(test.timestamp)

In [11]:
len(train), len(test)

(20216100, 41697600)

#### Reading weather data

In [12]:
weather_train = pd.read_csv("gs://123test_bucket/weather_train.csv")

In [13]:
weather_test = pd.read_csv("gs://123test_bucket/weather_test.csv")

In [14]:
meta = pd.read_csv("gs://123test_bucket/building_metadata.csv")

In [15]:
weather_train.timestamp = pd.to_datetime(weather_train.timestamp)

weather_test.timestamp = pd.to_datetime(weather_test.timestamp)

weather_train['month'] = weather_train.timestamp.dt.month
weather_train['dayofweek'] = weather_train.timestamp.dt.dayofweek
weather_train['hour'] = weather_train.timestamp.dt.hour

weather_test['month'] = weather_test.timestamp.dt.month
weather_test['dayofweek'] = weather_test.timestamp.dt.dayofweek
weather_test['hour'] = weather_test.timestamp.dt.hour

In [16]:
train_meta_w = pd.merge(weather_train, meta, on='site_id')
test_meta_w = pd.merge(weather_test, meta, on='site_id')

#### Scaling Data

In [17]:
train_meta_w = train_meta_w.fillna(0)
test_meta_w = test_meta_w.fillna(0)

In [18]:
scaler2 = MinMaxScaler()
num_cols = ['air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'square_feet', 'floor_count']
scaler2.fit(train_meta_w[num_cols])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
train_meta_w[num_cols] = scaler2.transform(train_meta_w[num_cols])
test_meta_w[num_cols] = scaler2.transform(test_meta_w[num_cols])

#### Encoding Categorical Data

In [ ]:
enc = OrdinalEncoder()
cat_cols = ['site_id', 'hour', 'dayofweek', 'month', 'primary_use', 'year_built']
other_cols = ['building_id', 'meter']
enc.fit(train_meta_w[cat_cols])

In [ ]:
train_meta_w[cat_cols] = enc.transform(train_meta_w[cat_cols])
test_meta_w[cat_cols] = enc.transform(test_meta_w[cat_cols])

#### Joining Weather, Metadata with load data

In [ ]:
len(train), len(train_meta_w)

In [ ]:
train = pd.merge(train, train_meta_w, on=['building_id', 'timestamp'], how='inner') # we have enough training data
train_meta_w = None
test = pd.merge(test, test_meta_w, on=['building_id', 'timestamp'], how='left') # we can't lose any rows
test_meta_w = None

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
len(train), len(test)

In [ ]:
train.columns

In [ ]:
train.head(2)

#### Random ID selection

Using 30% of the data as validation data. If more data is needed, we should consider adding data by randomly selecting buildings.

In [ ]:
ids = train['building_id'].unique()

In [ ]:
# keeping only 5% of the ids
len_sub = round(len(ids)*0.05)
ids_sub = np.random.choice(ids, len_sub, replace = False)

In [ ]:
# 0.3 represents the percentage of data that is kept for validation
len_val = round(len(ids_sub)*0.3)

In [ ]:
ids_val = np.random.choice(ids_sub, len_val, replace = False)

In [ ]:
ids_train = np.setdiff1d(ids_sub,ids_val)

In [ ]:
assert len(ids_val)+len(ids_train)==len(ids_sub)

#### Applying TimeseriesGenerator to the ASHRAE training data

Once difference between the example above, and our situation is that we have multiple timeseries, for each building and each meter in the building. So, we will be required to modify the code a little bit.

Below, we check how many meters exist in the dataset.

In [ ]:
len(train[['building_id', 'meter']].drop_duplicates()), len(test[['building_id', 'meter']].drop_duplicates())

The code below has been taken from this stackoverflow answer with some modifications:
https://stackoverflow.com/questions/55116638/use-keras-timeseriesgenerator-function-to-generate-squence-group-by-some-id/55118459#55118459

The modification is basically that once we subset the data for building ID, it is then subset for meter type also.

Further reading about modifying keras generator classes can be found below:
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [ ]:
col_names = num_cols + cat_cols + other_cols

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dt, col_names, batch_size = 10):
        self.batch_size = batch_size
        self.len = len(dt)//batch_size
        self.dt = dt[col_names + ['meter_reading']]
        self.col_names = col_names

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index_start = index*self.batch_size
        index_end = index_start+self.batch_size
        adf = self.dt[index_start:index_end]
        return adf[self.col_names], adf['meter_reading']

In [ ]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, dt, col_names, batch_size = 10):
        self.batch_size = batch_size
        self.len = len(dt)//batch_size
        self.dt = dt[col_names]
        self.col_names = col_names

    def __len__(self):
        return self.len+1

    def __getitem__(self, index):
        index_start = index*self.batch_size
        index_end = index_start+self.batch_size
        adf = self.dt[index_start:index_end]
        return adf[self.col_names]

#### Training and Validation Generators

In [ ]:
# Test
batch_size = 1024#1024+512
train_gen = DataGenerator(train[train['building_id'].isin(ids_train)], col_names=col_names, batch_size=batch_size)
val_gen = DataGenerator(train[train['building_id'].isin(ids_val)], col_names=col_names, batch_size=batch_size)   


In [ ]:
check1 = train_gen[0]
check2 = val_gen[0]

In [ ]:
check1[0].shape, check1[1].shape, len(col_names)

#### Loss Function - Root Mean Square

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

#### Dense NN

In [ ]:
#model = Sequential()
filters = 10
ncols = len(col_names) # categorical columns
states = 20
act = 'linear'

input_shape1 = Input(shape=(ncols,))

m1 = Dense(10, activation=act)(input_shape1)
m1 = Dense(5, activation=act)(m1)
m1 = LeakyReLU(alpha=0.1)(m1)
m1 = Dropout(0.2)(m1)
m1 = Dense(1, activation='sigmoid')(m1)

model2 = Model(inputs = input_shape1, outputs = m1)
model2._make_predict_function()

In [ ]:
opt = keras.optimizers.RMSprop(clipnorm=1.)

model2.compile(loss=root_mean_squared_error, optimizer=opt, metrics=['mse', 'mae', 'mape'])

In [ ]:
plot_model(model2)

In [ ]:
model2.summary()

In [ ]:
max(train['meter_reading'])

In [ ]:
# Train model on dataset
epochs = 1 #TODO: change it back to 5
workers = 10
model2.fit_generator(generator=train_gen,
                    validation_data=val_gen, epochs=1, 
                    use_multiprocessing=True, workers = workers)

#### Defining Function for Prediction - Method 1

In [ ]:
test_gen = TestDataGenerator(test, col_names=col_names, batch_size=batch_size)

In [ ]:
len(test_gen)

In [ ]:
result = model2.predict_generator(generator=test_gen, workers=workers, use_multiprocessing=True)

In [48]:
result

array([[0.00033766],
       [0.00042102],
       [0.00086147],
       ...,
       [0.0333986 ],
       [0.02364207],
       [0.02324256]], dtype=float32)

In [49]:
result.shape

(41498571, 1)

In [50]:
pd.Series(np.expm1(scaler1.inverse_transform(result)).reshape(result.shape[0],))

0             0.005724
1             0.007142
2             0.014667
3             0.008076
4             0.041769
               ...    
41498566     84.306885
41498567    142.774689
41498568      0.758586
41498569      0.491233
41498570      0.481197
Length: 41498571, dtype: float32

In [51]:
result_df = pd.DataFrame(pd.Series(np.expm1(scaler1.inverse_transform(result)).reshape(result.shape[0],)))

In [52]:
result_df.to_csv("result.csv", index=False)

In [53]:
len(pd.read_csv("gs://123test_bucket/test.csv"))

41697600

In [65]:
41697600/batch_size

40720.3125